In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [4]:
df=pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df.shape

(50000, 2)

In [7]:
df=df.sample(10000)

In [8]:
df.shape

(10000, 2)

In [9]:
df["review"]=df["review"].str.lower()

In [10]:
df.head()

,review,sentiment
12350,"i watched this film last night, i though i wou...",negative
30867,my wife and i are semi amused by howie mandel'...,negative
15880,if this is a 2008 product from one of the bigg...,negative
6653,"though a fan of shock and gore, i found this m...",negative
3116,only saw this movie late last night. i remembe...,negative


In [11]:
#remove html tags
import re
def tags(text):
    tag=re.compile("<.*?>")
    return re.sub(tag,"",text)

In [12]:
df["review"]=df["review"].apply(tags)

In [13]:
df.sample(5)

,review,sentiment
33452,"oh dear, oh dear, oh dear. my lifetime was dra...",negative
603,"worst dcom i have seen. ever. well, maybe not ...",negative
36386,raising victor vargas fails terribly in what i...,negative
20617,this film was just absolutly brilliant. it act...,positive
29812,i must confess to not having read the original...,positive


In [14]:
df["sentiment"]=df["sentiment"].map({"positive":0,"negative":1})

In [15]:
df.sample(5)

,review,sentiment
18716,"when me and my gf went to see this film, we di...",1
27325,milo is an overlooked & underrated horror flic...,0
36837,do not watch this movie if you loved the class...,1
18310,what a terrible sequel. the reason i give this...,1
398,"after a snowstorm, the roads are blocked and t...",1


In [16]:
import string

In [17]:
exclude=string.punctuation

In [18]:
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [19]:
def mark(text):
    for char in exclude:
        text=text.replace(char,"")
    return text

In [20]:
df["review"]=df["review"].apply(mark)

In [21]:
import nltk

In [22]:
from nltk.corpus import stopwords

In [23]:
def remove_stopword(text):
    x=[]
    for i in text.split():
        if i not in stopwords.words("english"):
            x.append(i)
            
    y=x[:]
    x.clear()
    return y
    

In [25]:
df["review"]=df["review"].apply(remove_stopword)

In [26]:
df

,review,sentiment
12350,"[watched, film, last, night, though, would, re...",1
30867,"[wife, semi, amused, howie, mandels, show, als...",1
15880,"[2008, product, one, biggest, production, hous...",1
6653,"[though, fan, shock, gore, found, movie, disap...",1
3116,"[saw, movie, late, last, night, remember, hype...",1
...,...,...
34328,"[reviewer, variety, said, overwritten, overlon...",1
5295,"[could, ever, hope, jackass, fanas, always, kn...",0
48137,"[think, 3, years, ago, saw, movie, accidentall...",0
440,"[movie, lame, funny, plot, doesnt, even, make,...",1


In [27]:
from nltk.stem.porter import PorterStemmer

In [28]:
ps=PorterStemmer()

In [29]:
y=[]
def stem(text):
    
    for i in text:
        y.append(ps.stem(i))
        
    z=y[:]
    y.clear()
    return z

In [30]:
df["review"]=df["review"].apply(stem)

In [31]:
df

,review,sentiment
12350,"[watch, film, last, night, though, would, rent...",1
30867,"[wife, semi, amus, howi, mandel, show, also, l...",1
15880,"[2008, product, one, biggest, product, hous, i...",1
6653,"[though, fan, shock, gore, found, movi, disapp...",1
3116,"[saw, movi, late, last, night, rememb, hype, r...",1
...,...,...
34328,"[review, varieti, said, overwritten, overlong,...",1
5295,"[could, ever, hope, jackass, fana, alway, knox...",0
48137,"[think, 3, year, ago, saw, movi, accident, rev...",0
440,"[movi, lame, funni, plot, doesnt, even, make, ...",1


In [32]:
def join_back(text):
    return " ".join(text)

In [33]:
df["review"]=df["review"].apply(join_back)

In [34]:
df

,review,sentiment
12350,watch film last night though would rent horror...,1
30867,wife semi amus howi mandel show also like shat...,1
15880,2008 product one biggest product hous indian f...,1
6653,though fan shock gore found movi disappoint sa...,1
3116,saw movi late last night rememb hype releas ho...,1
...,...,...
34328,review varieti said overwritten overlong kind ...,1
5295,could ever hope jackass fana alway knoxvil cre...,0
48137,think 3 year ago saw movi accident revisit inf...,0
440,movi lame funni plot doesnt even make sens sci...,1


In [35]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [36]:
x=df["review"].values

In [62]:
from sklearn.model_selection import GridSearchCV

In [83]:
# cv=CountVectorizer(max_features=1000)
tfid=TfidfVectorizer(max_features=3000)

In [84]:
# x=cv.fit_transform(df["review"]).toarray()
x1=tfid.fit_transform(df["review"]).toarray()

In [85]:
x.shape

(10000, 1000)

In [40]:
y=df.iloc[:,-1].values

In [41]:
y.shape

(10000,)

In [86]:
from sklearn.model_selection import train_test_split

In [87]:
x_train, x_test, y_train, y_test=train_test_split(x1,y,test_size=0.2)

In [88]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB

In [89]:
gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()

In [90]:
gnb.fit(x_train,y_train)
mnb.fit(x_train,y_train)
bnb.fit(x_train,y_train)

BernoulliNB()

In [91]:
y_pred1=gnb.predict(x_test)
y_pred2=mnb.predict(x_test)
y_pred3=bnb.predict(x_test)

In [92]:
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score

In [93]:
print("Gaussain",accuracy_score(y_test,y_pred1))
print("Multinomial",accuracy_score(y_test,y_pred2))
print("Bernoulli",accuracy_score(y_test,y_pred3))

Gaussain 0.7545
Multinomial 0.8345
Bernoulli 0.84


In [94]:
print("Gaussain",confusion_matrix(y_test,y_pred1))
print("Multinomial",confusion_matrix(y_test,y_pred2))
print("Bernoulli",confusion_matrix(y_test,y_pred3))

Gaussain [[730 270]
 [221 779]]
Multinomial [[819 181]
 [150 850]]
Bernoulli [[832 168]
 [152 848]]


In [95]:
print("Gaussain",precision_score(y_test,y_pred1))
print("Multinomial",precision_score(y_test,y_pred2))
print("Bernoulli",precision_score(y_test,y_pred3))

Gaussain 0.7426120114394662
Multinomial 0.8244422890397672
Bernoulli 0.8346456692913385


In [96]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [97]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)

In [98]:
clfs = {
    'SVC' : svc,
    'KN' : knc, 
    'NB': mnb, 
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [99]:
def train_classifier(clf,x_train,y_train,x_test,y_test):
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    
    return accuracy,precision

In [ ]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():
    
    current_accuracy,current_precision = train_classifier(clf, x_train,y_train,x_test,y_test)
    
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

For  SVC
Accuracy -  0.866
Precision -  0.8704453441295547
For  KN
Accuracy -  0.7085
Precision -  0.7192429022082019
For  NB
Accuracy -  0.8345
Precision -  0.8244422890397672
For  DT
Accuracy -  0.6895
Precision -  0.8101472995090017
For  LR
Accuracy -  0.8635
Precision -  0.8660624370594159
For  RF
Accuracy -  0.823
Precision -  0.8185404339250493
For  AdaBoost
Accuracy -  0.8065
Precision -  0.8169596690796277
